In [29]:
!pip install pyspark findspark

!pip install pybtex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Leitura das bibliotecas

In [30]:
from google.colab import files
import findspark
from pyspark.sql import SparkSession 
import os
import re
import hashlib
import yaml
import json
import csv
import requests
from numpy import nan
from pybtex.database.input import bibtex
from pybtex.database import BibliographyData, Entry
from unicodedata import normalize
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import math
import random
import numpy as np
from google.colab import drive 

spark = SparkSession.builder.appName('SparkRDDExamples').getOrCreate()



drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Leitura dos arquivos csv

In [31]:
arquivo_bib = spark.read\
.option("header", True)\
.option("sep","§" )\
.csv("/content/drive/MyDrive/Trabalho_Pyspark/output.csv")

In [32]:
jsc = spark.read\
.option("header", True)\
.option("sep",";" )\
.csv("/content/drive/MyDrive/Trabalho_Pyspark/jcs_2020.csv")

In [33]:
scimagojr = spark.read\
.option("header", True)\
.option("sep",";" )\
.csv("/content/drive/MyDrive/Trabalho_Pyspark/scimagojr 2020.csv")

In [34]:
arquivo_bib.count()

6031

In [35]:
arquivo_bib.show()


+---+--------------------+-----------------+--------------------+---------+--------------+--------------------+-----+------+--------+---------+--------------------+--------------------+--------------------+------+----+--------------------+-------------+--------------------+--------------------+----------+---------+----+-------+----------------+--------------------+
|_c0|            abstract|          address|                 doi|     issn|    issue_date|             journal|month|number|numpages|    pages|           publisher|               title|                 url|volume|year|           booktitle|         isbn|            keywords|            location|    series|articleno|note|edition|type_publication|              author|
+---+--------------------+-----------------+--------------------+---------+--------------+--------------------+-----+------+--------+---------+--------------------+--------------------+--------------------+------+----+--------------------+-------------+-----------

In [36]:
scimagojr.show()

+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+
|Rank|   Sourceid|               Title|   Type|              Issn|   SJR|SJR Best Quartile|H index|Total Docs. (2020)|Total Docs. (3years)|Total Refs.|Total Cites (3years)|Citable Docs. (3years)|Cites / Doc. (2years)|Ref. / Doc.|       Country|          Region|           Publisher|            Coverage|          Categories|
+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+
|   1|      28773|Ca-A Ca

In [37]:
jsc.show()

+----+--------------------+-----------+----+---------------------+-----------------+----+----+
|Rank|  Full Journal Title|Total Cites| _c3|Journal Impact Factor|Eigenfactor Score| _c6| _c7|
+----+--------------------+-----------+----+---------------------+-----------------+----+----+
|   1|CA-A CANCER JOURN...|     55,868|null|              508.702|         0.105140|null|null|
|   2|NATURE REVIEWS MO...|     58,477|null|               94.444|         0.075480|null|null|
|   3|NEW ENGLAND JOURN...|    464,351|null|               91.245|         0.631180|null|null|
|   4|NATURE REVIEWS DR...|     41,989|null|               84.694|         0.048220|null|null|
|   5|              LANCET|    369,601|null|               79.321|         0.445240|null|null|
|   6|Nature Reviews Cl...|     17,973|null|               66.675|         0.038760|null|null|
|   7|Nature Reviews Ma...|     19,887|null|               66.308|         0.056770|null|null|
|   8|       Nature Energy|     28,166|null|      

In [38]:
scimagojr.show()

+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+
|Rank|   Sourceid|               Title|   Type|              Issn|   SJR|SJR Best Quartile|H index|Total Docs. (2020)|Total Docs. (3years)|Total Refs.|Total Cites (3years)|Citable Docs. (3years)|Cites / Doc. (2years)|Ref. / Doc.|       Country|          Region|           Publisher|            Coverage|          Categories|
+----+-----------+--------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+
|   1|      28773|Ca-A Ca

#Tratando as chaves dos arquivos

In [39]:
scimagojr = scimagojr.withColumn('Title_Hash', sha1(scimagojr['Title']))
scimagojr.show(truncate=False)

+----+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+------------------------------------------------------------+-------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------+
|Rank|Sourceid   |Title                                                                                                                                                           |Type   |Issn              |SJR   |SJR Best Quartile|H index|Total Docs. (2020)|Total Docs. (3years)|Tota

In [40]:
jsc = jsc.withColumn('Full Journal Title', lower(jsc['Full Journal Title']))
jsc = jsc.withColumn('Title_Hash', sha1(lower(jsc['Full Journal Title'])))
jsc.show(truncate=False)

+----+------------------------------------------------+-----------+----+---------------------+-----------------+----+----+----------------------------------------+
|Rank|Full Journal Title                              |Total Cites|_c3 |Journal Impact Factor|Eigenfactor Score|_c6 |_c7 |Title_Hash                              |
+----+------------------------------------------------+-----------+----+---------------------+-----------------+----+----+----------------------------------------+
|1   |ca-a cancer journal for clinicians              |55,868     |null|508.702              |0.105140         |null|null|8baaabd0ff5fa35f17411380ad5728d74011870c|
|2   |nature reviews molecular cell biology           |58,477     |null|94.444               |0.075480         |null|null|0543823fdbadf8093eb9e6aad44251d03d3c627e|
|3   |new england journal of medicine                 |464,351    |null|91.245               |0.631180         |null|null|04f1786be8d2a3aa27752e400aec1f0fc3845d9f|
|4   |nature rev

In [41]:
arquivo_bib = arquivo_bib.withColumn('issn', regexp_replace('issn','-', ''))

In [42]:
arquivo_bib.show()

+---+--------------------+-----------------+--------------------+--------+--------------+--------------------+-----+------+--------+---------+--------------------+--------------------+--------------------+------+----+--------------------+-------------+--------------------+--------------------+----------+---------+----+-------+----------------+--------------------+
|_c0|            abstract|          address|                 doi|    issn|    issue_date|             journal|month|number|numpages|    pages|           publisher|               title|                 url|volume|year|           booktitle|         isbn|            keywords|            location|    series|articleno|note|edition|type_publication|              author|
+---+--------------------+-----------------+--------------------+--------+--------------+--------------------+-----+------+--------+---------+--------------------+--------------------+--------------------+------+----+--------------------+-------------+--------------

In [43]:
arquivo_bib.count()

6031

In [44]:
scimagojr = scimagojr.withColumn('issn', explode(split(scimagojr.Issn, ",")))

In [45]:
scimagojr.show()

+----+-----------+--------------------+-------+---------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+--------------------+
|Rank|   Sourceid|               Title|   Type|     issn|   SJR|SJR Best Quartile|H index|Total Docs. (2020)|Total Docs. (3years)|Total Refs.|Total Cites (3years)|Citable Docs. (3years)|Cites / Doc. (2years)|Ref. / Doc.|       Country|          Region|           Publisher|            Coverage|          Categories|          Title_Hash|
+----+-----------+--------------------+-------+---------+------+-----------------+-------+------------------+--------------------+-----------+--------------------+----------------------+---------------------+-----------+--------------+----------------+--------------------+--------------------+--------------------+-----------

#Join das bases envolvidas

In [48]:
dfjournalrank = jsc.join(scimagojr, on='Title_Hash', how='outer')

In [49]:
df_join_bibtex_csv = arquivo_bib.join(dfjournalrank, on='issn', how='left')

# Drop colunas que não são necessárias 

In [59]:
df_join_bibtex_csv.columns

['issn',
 'abstract',
 'address',
 'doi',
 'issue_date',
 'journal',
 'month',
 'number',
 'numpages',
 'pages',
 'url',
 'volume',
 'year',
 'booktitle',
 'isbn',
 'keywords',
 'location',
 'series',
 'articleno',
 'note',
 'edition',
 'type_publication',
 'author',
 'Title_Hash',
 'Full Journal Title',
 'Total Cites',
 'Journal Impact Factor',
 'Eigenfactor Score',
 'Sourceid',
 'Type',
 'SJR',
 'SJR Best Quartile',
 'H index',
 'Total Docs. (2020)',
 'Total Docs. (3years)',
 'Total Refs.',
 'Total Cites (3years)',
 'Citable Docs. (3years)',
 'Cites / Doc. (2years)',
 'Ref. / Doc.',
 'Country',
 'Region',
 'Coverage',
 'Categories']

In [58]:
columns = ['_c0','Rank','title', '_c3', '_c7', '_c6', 'publisher']
df_join_bibtex_csv = df_join_bibtex_csv.drop(*(columns))


# Escrita do resultado em csv

In [56]:
df_join_bibtex_csv.write.csv('/content/drive/MyDrive/Trabalho_Pyspark/resultado_pyspark.csv')
